<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">M5.4 – شفافة خوارزمية كتابة</h1><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M5-Open-Science-for-Crops" dir="ltr">Open Climate Science for Crops &amp; Crop Conditions</a></p>
</div>

In [ ]:
import datetime
import glob
import numpy as np
import xarray as xr
import rasterio
import rioxarray
from matplotlib import pyplot
from rasterio.warp import Resampling

DATE_START = datetime.date(2023, 10, 1)
DATE_SOWING = datetime.date(2023, 10, 15)
DATE_END = datetime.date(2024, 9, 29)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نظرة عامة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا الآن جاهزين لحساب مؤشر رضا متطلبات المياه <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>. ولأن <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> خوارزمية متوسطة التعقيد، فمن المهم أن نكتب شيفرة تحقق المعايير التالية:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تطبيق خوارزمية <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> بشكل صحيح، بالطبع.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أن يتمكن أي قارئ من فهم ما تقوم به كل سطر من الشيفرة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أن يرتبط كل جزء من الشيفرة بوضوح بجزء محدد من الوصف المكتوب للخوارزمية (عند الاقتضاء).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أن تكون الوحدات الفيزيائية للمدخلات والمخرجات والمتغيرات الوسيطة المهمة واضحة.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذا الدرس سنرى أمثلة على شيفرة تحقق هذه المعايير، مما يجعل قراءتها والتحقق منها أسهل للبشر.</p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">«الصحيح» معنى</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">من الضروري أن تنفذ شيفرتنا الحاسوبية الخوارزمية أو النموذج الذي نعمل عليه <strong>تنفيذًا صحيحًا</strong>. نقصد بـ «الصحيح» أن الشيفرة تفعل ما هو مقصود، وليس بالضرورة أن تكون النتائج دقيقة مقارنة بحالة موضوعية أو قياس واقعي.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدقة مهمة، لكنها ثانوية بالنسبة لصحة التنفيذ. فإذا كان نموذجنا <strong>غير صحيح</strong> و<strong>غير دقيق</strong>، فلن نستطيع تحسين الدقة قبل إصلاح التنفيذ. تخيّل أنك تحاول معرفة هل هناك مشكلة في أضواء السيارة الخلفية: تطلب من صديقك تشغيل إشارة الانعطاف اليمنى لكنه يشغل اليسرى—حينها المشكلة ليست في الضوء نفسه، بل في تنفيذ التعليمات.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">PET</span> بيانات تجميع</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الدرس السابق أسقطنا بيانات النتح-التبخر <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span> ثم جمعناها في كائن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> واحد. لنفتح ذلك الملف، مع خريطة تصنيف الغطاء الأرضي.</p>
</div>

In [ ]:
with rasterio.open('data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff') as dataset:
    lc_map = dataset.read(1)

ds_et = xr.open_dataset('data/processed/VNP16_ET_and_PET_mm_day-1.nc4')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك، وبما أن بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> مبنية على مركبات <span dir="ltr" style="unicode-bidi: isolate;">8</span>-أيام، فلنستوفيها إلى خطوات زمنية يومية باستخدام الدالة <span dir="ltr" style="unicode-bidi: isolate;">resample()</span> في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>. نستدعي <span dir="ltr" style="unicode-bidi: isolate;">ffill()</span> بعد <span dir="ltr" style="unicode-bidi: isolate;">resample()</span> لملء القيم المفقودة بين كل خطوة <span dir="ltr" style="unicode-bidi: isolate;">8</span>-أيام عبر نسخ القيمة السابقة للأمام (ملء أمامي).</p>
</div>

In [ ]:
ds_et_daily = ds_et.where(lc_map == 1, np.nan).resample(time = '1D').ffill()
ds_et_daily

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الأولية التربة ماء محتوى حساب</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">آخر مدخل نحتاجه لحساب <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> هو المحتوى الأولي لماء التربة عند الخطوة الزمنية <span dir="ltr" style="unicode-bidi: isolate;">t=0</span>. وبما أن <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> حساب ميزانية مائية—<a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">مشابه لما فعلناه في الوحدة 3</a>—فنحتاج لمعرفة كمية الماء الموجودة ابتدائيًا في «دلو» ماء التربة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنقرّب ذلك بافتراض أن الهطول و<span dir="ltr" style="unicode-bidi: isolate;">ET</span> يتوازنان على مقياس سنوي؛ أي بإهمال الجريان السطحي نفترض:</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">$$
\sum^{\text{1 year}} \text{Soil water} = \sum^{\text{1 year}} \text{Precipitation} - \sum^{\text{1 year}} \text{ET}
$$</span></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هذا افتراض خشن، لكن يتبين أن <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> غير حساس جدًا لقيمة <span dir="ltr" style="unicode-bidi: isolate;">SWC</span> الابتدائية.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لحساب ماء التربة الابتدائي نحتاج إلى تحميل بيانات الهطول. في الدرس السابق أسقطنا بيانات <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> على شبكة <span dir="ltr" style="unicode-bidi: isolate;">EASE-Grid 2.0</span> بدقة <span dir="ltr" style="unicode-bidi: isolate;">9-km</span> لتقليل المتطلبات التخزينية والحسابية. الآن نحتاج إلى إعادة أخذ العينات مرة أخرى لتطابق دقة <span dir="ltr" style="unicode-bidi: isolate;">1-km</span> الخاصة ببيانات <span dir="ltr" style="unicode-bidi: isolate;">ET/PET</span>.</p>
</div>

In [ ]:
# NOTE: This might take several seconds

# Resample the precip data to match our ET data
ds_precip = xr.open_dataset('data/processed/IMERG_precip_mm_day-1_for_study_area.nc4')\
    .rio.write_crs(ds_et.rio.crs)
ds_precip_1km = ds_precip.rio.reproject_match(ds_et, resampling = Resampling.bilinear)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أن لدينا سجل هطول بطول <span dir="ltr" style="unicode-bidi: isolate;">366</span> يومًا.</p>
</div>

In [ ]:
ds_precip_1km.time.size

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ولا حظ أيضًا أن ذلك يطابق بيانات <span dir="ltr" style="unicode-bidi: isolate;">ET</span> و<span dir="ltr" style="unicode-bidi: isolate;">PET</span> اليومية بعد الاستيفاء.</p>
</div>

In [ ]:
ds_et_daily.time.size

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لذلك يمكننا حساب مجموع الهطول السنوي (على دورة سنوية بطول <span dir="ltr" style="unicode-bidi: isolate;">366</span> يومًا) بأخذ المجموع عبر البعد <span dir="ltr" style="unicode-bidi: isolate;">&#x27;time&#x27;</span>:</p>
</div>

In [ ]:
precip_total = ds_precip_1km.sum('time').precipitation

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نقوم بالشيء نفسه مع <span dir="ltr" style="unicode-bidi: isolate;">ET</span> ثم نطرحها من متوسط الهطول السنوي. النتيجة هي كمية الماء المتوقعة (مم) التي يمكن أن تُخزن في التربة. وللحصول على مقدار يومي مناسب نقسم هذا الباقي السنوي على عدد أيام السنة (<span dir="ltr" style="unicode-bidi: isolate;">366</span>).</p>
</div>

In [ ]:
# Our best guess for the soil water content might be
#    the mean annual balance between precip and ET

swc = (precip_total - ds_et_daily.ET.sum('time')) / ds_precip_1km.time.size # (366 days)
swc

In [ ]:
swc.where(swc >= 0, 0).where(lc_map == 1, np.nan).plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المحصول معاملات و الجذور عمق و الحقل سعة</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في كثير من النماذج العلمية توجد معاملات يجب تحديدها اعتمادًا على قياسات ميدانية أو الأدبيات أو النظرية. في <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> توجد معاملات تمثل تأثير خصائص التربة على ميزانية ماء التربة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يُعطى المستوى الحرج لماء التربة <span dir="ltr" style="unicode-bidi: isolate;">SWC</span> بالعلاقة:</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">$$
\text{SWC}(t) = \text{FC}\times \text{SW}_f\times \text{RD}_f(t)
$$</span></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تعتمد سعة الحقل <span dir="ltr" style="unicode-bidi: isolate;">FC</span> على قوام التربة. وللتبسيط سنفترض تربة طميية رملية بقيمة وسطية من <a href="https://www.fao.org/4/X0490E/x0490e0c.htm#soil%20evaporation%20reduction%20coefficient%20(kr)" dir="ltr">FAO 56 (الجدول 19)</a> مقدارها <span dir="ltr" style="unicode-bidi: isolate;">0.23 m^3 m^-3</span>. إذا كان عمق الجذور الأقصى للقمح نحو <span dir="ltr" style="unicode-bidi: isolate;">1.5 m</span> (<a href="https://doi.org/10.1016/j.fcr.2016.02.013" dir="ltr">Fan وآخرون 2016</a>) فإن ذلك يقابل:</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">$$
\text{FC} = \frac{0.23\, \text{m}^3\text{m}^{-3}}{1.5\,\text{m}\times 1.0\,\text{m} \times 1.0\,\text{m}} = 0.153 \,\text{m} = 153 \,\text{mm}
$$</span></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبالرجوع إلى <span dir="ltr" style="unicode-bidi: isolate;">FAO 56</span> لخصائص تربة تمثيلية، وفقًا لـ <a href="https://www.fao.org/4/X0490E/x0490e0e.htm#readily%20available%20water%20(raw)" dir="ltr">FAO 56 (الجدول 22)</a> للقمح الربيعي، <span dir="ltr" style="unicode-bidi: isolate;">SW_f = 1 - 0.55 = 0.45</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يتغير <span dir="ltr" style="unicode-bidi: isolate;">RD_f</span> عبر الزمن مع تطور الجذور. بالنسبة للقمح في شمال الجزائر يحدث البذر عادة في أكتوبر أو نوفمبر، ويصل المحصول إلى النضج بين فبراير وأبريل. وبغياب معلومات الحقول الفردية سنستخدم افتراضات مبسطة كما في الدرس.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المحصول معامل حساب</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحتاج إلى تحويل معاملات المحصول إلى سلسلة زمنية يومية لتتوافق مع خوارزميتنا ذات الخطوة الزمنية اليومية. الدالة <span dir="ltr" style="unicode-bidi: isolate;">linear_interp()</span> أدناه تُنشئ سلسلة زمنية يومية كاملة انطلاقًا من عدد محدود من الأيام ذات قيم معلومة.</p>
</div>

In [ ]:
def linear_interp(doy, known_values, known_doy):
    '''
    Linearly interpolates between coefficients on known dates.

    Parameters
    ----------
    doy : Sequence
        Integer sequence of day-of-year
    known_values : Sequence
        The known values (on specific days)
    known_doy : Sequence
        The day-of-year corresponding to each value in `known_values`

    Returns
    -------
    Sequence
    '''
    # Place the crop coefficients into the empty DOY time series
    arr = np.nan * np.ones(doy.shape)
    arr[np.in1d(doy, known_doy)] = known_values
    
    # Get the ordinal positions of each DOY value for which we know
    #    the crop coefficient
    x_loc = np.argwhere(np.in1d(doy, known_doy)).ravel()
    # Interpolate between the known crop coefficients
    return np.interp(np.arange(0, doy.size), x_loc, known_values)

In [ ]:
# We linearly interpolate the crop coefficients

doy_start = int(DATE_START.strftime('%j'))
doy_k0 = int(DATE_SOWING.strftime('%j')) # Planting date
doy_k1 = int(datetime.date(2024, 4, 1).strftime('%j')) # Mature date
doy_k2 = int(datetime.date(2024, 6, 1).strftime('%j')) # Harvest date
doy_end = int(DATE_END.strftime('%j')) # To have a complete year
crop_coef_known = np.array([0.3, 1.15, 0.25])

# NOTE: Have to add a +1 to end because 2024 is a leap year
doy = np.concatenate([np.arange(doy_start, 366), np.arange(1, doy_end + 1)])
crop_coef = linear_interp(doy, crop_coef_known, [doy_k0, doy_k1, doy_k2])

# Get DOY of the planting date
doy.tolist().index(doy_k0)

# TODO Making sure that crop coef is zero prior to planting date
crop_coef[0:14] = 0

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات: الرقمية البيانات أنواع</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ينبغي دائمًا الانتباه إلى نوع البيانات الرقمية المستخدم لتخزين القيم. فزيادة عدد البتات تزيد الدقة لكنها تستهلك ذاكرة أكبر. معاملات المحصول يمكن غالبًا تمثيلها بأمان باستخدام <span dir="ltr" style="unicode-bidi: isolate;">16</span> بت فقط، وهو الحد الأدنى لعدد عشري («نقطة عائمة»). قارن المثالين أدناه.</p>
</div>

In [ ]:
# Note memory-saving potential of changing the dtype

print('64-bit precision:', crop_coef[20:30])
crop_coef = crop_coef.astype(np.float16)
print('16-bit precitions:', crop_coef[20:30])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الجذر كسر حساب</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك نحتاج إلى استيفاء سلسلة كسر الجذور بنفس الطريقة. نحن نعرف قيمة كسر الجذور في أيام محددة من السنة، لذلك نستخدم الاستيفاء الخطي بين تلك التواريخ. يوضح الشكل النهائي أدناه ذلك إلى جانب معامل المحصول.</p>
</div>

In [ ]:
root_fraction = linear_interp(doy, [0.1, 1.0], [doy_k0, doy_k1])
root_fraction = root_fraction.astype(np.float16)
root_fraction[0:14] = 0.0

# Get human-readable date labels
date_labels = [
    datetime.datetime.strptime('2023-%03d' % d, '%Y-%j') if d >= doy_start\
    else datetime.datetime.strptime('2024-%03d' % d, '%Y-%j')
    for d in doy
]
pyplot.ylabel('Crop Coefficient or Root Fraction')
pyplot.plot(date_labels, crop_coef, 'k-', label = 'Crop Coefficient')
pyplot.plot(date_labels, root_fraction, 'r-', label = 'Root Fraction')
pyplot.legend()
pyplot.show()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمثل معامل المحصول القدرة العامة للمحصول على امتصاص الماء من التربة ونقله (نتحه) إلى الغلاف الجوي، بينما يمثل كسر الجذور نسبة عمق الجذور التي يستكشفها النبات في مرحلته الحالية. كلاهما يزداد خلال موسم النمو حتى النضج، لأن زيادة كسر الجذور تعني أن كمية أكبر من ماء التربة يمكن استغلالها. عند النضج، بالنسبة لمحصول مثل القمح الربيعي، ينخفض معامل المحصول بسرعة لأن النبات يصبح مركزًا على بناء الحبوب.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الأولية نسخة تنفيذ</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا أخيرًا جاهزين لبدء كتابة خوارزمية <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>. خطوة أساسية هي تصميم <strong>الواجهة</strong>؛ أي شكل الدالة التي ستنفذ <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نصمم الواجهة أولًا لأن الخوارزمية جزء من سير عمل أكبر، ولأن الواجهة تحمل تفاصيل لا تعتمد على اختيار التنفيذ. وعمومًا أي خوارزمية لها:</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مدخلات مطلوبة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مخرجات مطلوبة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">خيارات أو معاملات تغيّر سلوك الخوارزمية.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تصميم واجهة في <span dir="ltr" style="unicode-bidi: isolate;">Python</span> يبدأ بتوقيع الدالة باستخدام <span dir="ltr" style="unicode-bidi: isolate;">def</span> وإضافة <span dir="ltr" style="unicode-bidi: isolate;">docstring</span>. مثلًا:</p>
</div>

In [ ]:
def water_requirement_satisfaction_index(
        doy, crop_coef, pet, precip, root_fraction, sw_init, 
        sw_frac = 0.45, field_capacity = 0.153):
    '''
    Computes the water requirement satisfaction index (WRSI), based on Senay
    and Verdin (2003):

        Senay, G. B., and J. Verdin. 2003. Characterization of yield 
        reduction in Ethiopia using a GIS-based crop water balance model. 
        Canadian Journal of Remote Sensing 29 (6):687–692.

    The default values for the `field_capacity` and the soil-water level 
    as a fraction of field capacity, `sw_frac`, are taken from the FAO 56
    manual, Tables 19 and 22, respectively, for Spring Wheat:

        Allen, R. G., L. S. Pereira, D. Raes, and M. Smith. 1998. 
        Crop evapotranspiration - Guidelines for computing crop water requirements. 
        Rome, Italy: FAO - Food and Agriculture Organization of the United Nations.
        
    
    Parameters
    ----------
    doy : Sequence
        The day of year (DOY) for each time step
    crop_coef : Sequence
        A sequence of crop coefficients for each time step
    pet : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    sw_frac : Number
        The soil water level, as a fraction of field capacity, below which
        AET becomes less than PET during the crop's mature stage; should be
        1.0 minus the allowable depletion fraction (dim.)
    field_capacity : Number
        The field capacity (mm)
    
    Returns
    -------
    Sequence
        The WRSI on each day of the simulation (mm day-1)
    '''
    pass

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>لاحظ أن:</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أدرجنا مراجع للاستدلال على معلومات خلفية أساسية لتبرير الخيارات التي اتخذناها.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وصفنا الوحدات الفيزيائية لكل الكميات.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عبارة <span dir="ltr" style="unicode-bidi: isolate;">pass</span> ليست ضرورية تمامًا لكنها مفيدة. لدينا الآن دالة لا تفعل شيئًا بعد، لكنها تحدد بوضوح المدخلات المطلوبة، والمخرجات المتوقعة (في <span dir="ltr" style="unicode-bidi: isolate;">docstring</span>)، والمعاملات الاختيارية وقيمها الافتراضية.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> خوارزمية</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فيما يلي تنفيذ واحد لخوارزمية <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>لاحظ أن:</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أضفنا عدة تعليقات لمساعدة القارئ على فهم الشيفرة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تتضمن التعليقات أحيانًا إشارة إلى <strong>الوصف المرجعي</strong> (مثل ورقة Senay & Verdin 2003) وتحدد أحيانًا أي معادلات تُنفّذها مقاطع معينة، مما يسهل التحقق من صحة التنفيذ.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وجود عدة عبارات <span dir="ltr" style="unicode-bidi: isolate;">assert</span> في بداية الشيفرة يساعد على تقنين توقعاتنا حول شكل بيانات الإدخال. إذا مرر المستخدم بيانات بصيغة غير صحيحة فسيحصل على رسالة خطأ مفيدة.</li>
</ul>
</div>

In [ ]:
def water_requirement_satisfaction_index(
        doy, crop_coef, pet, precip, root_fraction, sw_init, 
        sw_frac = 0.45, field_capacity = 0.153):
    '''
    Computes the water requirement satisfaction index (WRSI), based on Senay
    and Verdin (2003):

        Senay, G. B., and J. Verdin. 2003. Characterization of yield 
        reduction in Ethiopia using a GIS-based crop water balance model. 
        Canadian Journal of Remote Sensing 29 (6):687–692.

    The default values for the `field_capacity` and the soil-water level 
    as a fraction of field capacity, `sw_frac`, are taken from the FAO 56
    manual, Tables 19 and 22, respectively, for Spring Wheat:

        Allen, R. G., L. S. Pereira, D. Raes, and M. Smith. 1998. 
        Crop evapotranspiration - Guidelines for computing crop water requirements. 
        Rome, Italy: FAO - Food and Agriculture Organization of the United Nations.

        
    Parameters
    ----------
    doy : Sequence
        The day of year (DOY) for each time step
    crop_coef : Sequence
        A sequence of crop coefficients for each time step
    pet : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    sw_frac : Number
        The soil water level, as a fraction of field capacity, below which
        AET becomes less than PET during the crop's mature stage; should be
        1.0 minus the allowable depletion fraction (dim.)
    field_capacity : Number
        The field capacity (mm)
    
    Returns
    -------
    Sequence
        The WRSI on each day of the simulation (mm day-1)
    '''
    assert crop_coef.ndim == 1, 'crop_coef should be a 1-dimensional numeric sequence'
    assert pet.ndim == 1, 'pet should be a 1-dimensional numeric sequence'
    assert precip.ndim == 1, 'precip should be a 1-dimensional numeric sequence'
    assert root_fraction.ndim == 1, 'root_fraction should be a 1-dimensional numeric sequence'
    # Pre-allocate vectors for holding data for each time step
    paw = np.nan * np.ones(pet.shape, dtype = np.float32) # Plant available water
    sw  = np.nan * np.ones(pet.shape, dtype = np.float32) # Soil water
    aet_c = np.nan * np.ones(pet.shape, dtype = np.float32) # AETc
    # Compute PETc at each time step (Eq. 2, Senay & Verdin 2003)
    pet_c = pet * crop_coef # PETc
    # Compute the critical soil water at each time step
    #    (Eq. 7, Senay & Verdin 2003)
    sw_crit = field_capacity * sw_frac * root_fraction
    for t in range(len(pet)):
        # Compute plant available water (PAW) (Eq. 3, Senay & Verdin 2003)
        if t == 0:
            paw[t] = sw_init + precip[t] # At t=0, sw[t-1] is unknown
        else:
            paw[t] = sw[t-1] + precip[t]
        # Compute AET (Eq. 4-6, Senay & Verdin 2003); above the critical
        #    soil water level, this is just PET
        if paw[t] >= sw_crit[t]:
            aet_c[t] = pet_c[t]
        else:
            aet_c[t] = (paw[t] / sw_crit[t]) * pet_c[t]
        aet_c[t] = np.min([aet_c[t], paw[t]]) # Cannot be higher than PAW
        # Compute (remaining) soil water
        sw[t] = paw[t] - aet_c[t]
        sw[t] = np.min([sw[t], field_capacity]) # Cannot be higher than FC
        sw[t] = np.max([sw[t], 0.0])
    
    # At the end, compute WRSI on each day of the simulation; i.e.,
    #    compute totals on each day (a cumulative sum)
    wrsi = (np.cumsum(aet_c) / np.cumsum(pet_c)) * 100
    return wrsi.astype(np.float32)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كيف تعمل هذه الدالة؟ لنجربها على بيانات اصطناعية. يمكن أن يكون ذلك أساسًا لاختبار وحدة <span dir="ltr" style="unicode-bidi: isolate;">unit test</span> نستخدمه لاحقًا للتحقق من أن الدالة ما زالت تعمل كما هو مقصود.</p>
</div>

In [ ]:
# Random precipitation time series, between 0 and 3 mm
precip = np.random.choice([0, 1, 2, 3], size = doy.size, p = [0.6, 0.3, 0.08, 0.02])
pet = 3 * np.ones(doy.size) # Constant 3 mm of PET

wrsi = water_requirement_satisfaction_index(
    doy[14:], crop_coef[14:], pet[14:], precip[14:], root_fraction[14:], 
    sw_init = 10, sw_frac = 0.45, field_capacity = 0.153)

In [ ]:
pyplot.plot(date_labels[15:], wrsi[1:], 'k-')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المكان عبر دالتنا تطبيق</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخوارزميات التكرارية مثل <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> لا تعمل جيدًا مع <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لأنها ليست مجرد سلسلة عمليات جبرية. لذلك لتطبيق الخوارزمية على ملايين البكسلات في مجالنا المكاني، نحتاج طريقة للتوسع دون الاعتماد على <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> وتوافقه مع <span dir="ltr" style="unicode-bidi: isolate;">dask</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أحد الأساليب هو اعتبار كل بكسل عنصرًا مستقلًا في سلسلة كبيرة يمكن معالجته على حدة. لذلك سنعيد تشكيل بيانات الإدخال من خرائط ثلاثية الأبعاد إلى مجموعة سلاسل زمنية أحادية البعد.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مثلًا، بيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span> حاليًا على شكل <span dir="ltr" style="unicode-bidi: isolate;">(T, Y, X)</span> (مصفوفة ثلاثية الأبعاد).</p>
</div>

In [ ]:
pet = ds_et_daily.sel(time = slice(DATE_START, DATE_END)).PET.values
pet.shape

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نريد إعادة تشكيلها إلى مصفوفة ثنائية الأبعاد <span dir="ltr" style="unicode-bidi: isolate;">(T x N)</span>، حيث <span dir="ltr" style="unicode-bidi: isolate;">T</span> عدد الخطوات الزمنية و<span dir="ltr" style="unicode-bidi: isolate;">N</span> عدد البكسلات.</p>
</div>

In [ ]:
# Reshaping to a (T x N) array for N pixels
days, rows, cols = pet.shape
pet_raveled = pet.reshape((days, rows * cols))
pet_raveled.shape

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما نرى، يوجد نحو <span dir="ltr" style="unicode-bidi: isolate;">1.6</span> مليون بكسل في مجالنا المكاني. لكن العديد من هذه البكسلات يمكن تجاهله لأنه ماء أو غطاء أرضي ليس أراضي زراعية. هل يمكننا استخدام خريطة الغطاء الأرضي لتصفية البكسلات غير الزراعية؟</p>
</div>

In [ ]:
# Note that lc_map has same shape as our raveled spatial domain

lc_map.ravel().shape

In [ ]:
# Filtering the NoData pixels out using the land-cover map
pet_series = pet_raveled[:,lc_map.ravel() == 1]
pet_series.shape

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بما أن <span dir="ltr" style="unicode-bidi: isolate;">lc_map</span> لديه عدد العناصر (البكسلات) نفسه مثل بيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span>، يمكننا استخدامه لتصفية البيانات. <span dir="ltr" style="unicode-bidi: isolate;">65,000</span> بكسل أفضل بكثير من <span dir="ltr" style="unicode-bidi: isolate;">1.6</span> مليون!</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أدناه نكرر ذلك لبيانات الهطول وبيانات ماء التربة الابتدائي.</p>
</div>

In [ ]:
precip_series = ds_precip_1km.sel(time = slice(DATE_START, DATE_END)).precipitation.values\
    .reshape((days, rows * cols))
precip_series = precip_series[:,lc_map.ravel() == 1]
print(precip_series.shape)

swc_series = swc.where(swc >= 0, 0).values.reshape((rows * cols,))
swc_series = swc_series[lc_map.ravel() == 1]
print(swc_series.shape)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أصبحنا الآن جاهزين لحساب</strong> <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> <strong>لكل بكسلات الأراضي الزراعية.</strong></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد يستغرق الجزء التالي نحو <span dir="ltr" style="unicode-bidi: isolate;">6–8</span> دقائق.</p>
</div>

In [ ]:
from tqdm import tqdm

# Get index of first time step when crop is sown
t = doy.tolist().index(int(DATE_SOWING.strftime('%j')))

# Get a data structure to hold WRSI outputs
wrsi = np.nan * np.ones(precip_series[t:,].shape, dtype = np.float32)

for i in tqdm(range(precip_series.shape[1])):
    wrsi[:,i] = water_requirement_satisfaction_index(
        doy[t:], crop_coef[t:], pet_series[t:,i], precip_series[t:,i], 
        root_fraction[t:], sw_init = swc_series[i], sw_frac = 0.45, field_capacity = 0.153)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الحيز ثنائي الأبعاد إلى الأحادي النتائج تحويل</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قمنا بتصفية البكسلات إلى بكسلات الأراضي الزراعية فقط للحصول على سلاسل زمنية أحادية البعد أصغر. فكيف نستعيد التمثيل المكاني ثنائي الأبعاد الذي فقدناه؟</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الجواب: نستخدم <span dir="ltr" style="unicode-bidi: isolate;">lc_map</span> مرة أخرى ولكن بعكس العملية. ننشئ خريطة ثنائية الأبعاد فارغة (فعليًا مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">(T, Y, X)</span>) ثم نضع السلاسل الزمنية بطول <span dir="ltr" style="unicode-bidi: isolate;">T</span> في أماكنها باستخدام <span dir="ltr" style="unicode-bidi: isolate;">lc_map</span> لتحديد مواضع البيانات. وبما أن <span dir="ltr" style="unicode-bidi: isolate;">numpy</span> يستخدم الفهرسة بالطريقة نفسها دائمًا، يمكننا إعادة استخدام تعبير التصفية <span dir="ltr" style="unicode-bidi: isolate;">lc_map == 1</span> للإشارة إلى المواقع نفسها.</p>
</div>

In [ ]:
wrsi_map = np.nan * np.ones((wrsi.shape[0], *lc_map.shape), dtype = np.float32)
wrsi_map[:,lc_map == 1] = wrsi
wrsi_map.shape

In [ ]:
pyplot.imshow(wrsi_map[-1], interpolation = 'nearest', vmin = 0)
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا تحويل بيانات <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> إلى كائن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لتسهيل اقتطاع البيانات ورسمها.</p>
</div>

In [ ]:
ds_wrsi = xr.Dataset(
    data_vars = {'WRSI': (['time', 'y', 'x'], wrsi_map)},
    coords = {
        'time': ds_et_daily.time[(t+1):],
        'x': ds_et_daily.x,
        'y': ds_et_daily.y
    })

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنقم بالتقريب على منطقة تونس في شمال الجزائر وننظر إلى خريطة <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> في التاريخ الذي قدّرنا فيه وصول القمح الربيعي إلى النضج، <span dir="ltr" style="unicode-bidi: isolate;">April 1, 2024</span>. نلاحظ تدرجًا عامًا من الشمال إلى الجنوب، حيث تكون الأراضي الزراعية في الشمال أقرب إلى تلبية متطلبات المياه مقارنة بالجنوب. وهذا يتسق مع تدرج مكاني عام في الهطول و<span dir="ltr" style="unicode-bidi: isolate;">PET</span>.</p>
</div>

In [ ]:
ds_wrsi.sel(y = slice(4.45e6, 4.225e6), x = slice(0.8e6, 1e6), time = '2024-04-01').WRSI.plot(vmin = 0)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبما أننا أصبحنا نملك كائن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>، يمكننا أيضًا استخراج سلسلة زمنية لموقع محدد.</p>
</div>

In [ ]:
tunis_coords = {'x': 927211, 'y': 4414719}

ds_wrsi.sel(**tunis_coords, method = 'nearest').WRSI.plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تونس و تيارت مقارنة</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا مقارنة <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> في موقعين مختلفين مثل تونس وتيارت. سنرى في الشكل أدناه أنه رغم أن الموقعين يختبران تذبذبًا متشابهًا في الهطول و<span dir="ltr" style="unicode-bidi: isolate;">PET</span>، إلا أن تونس عادة أكثر رطوبة من تيارت، ولذلك يكون لديها <span dir="ltr" style="unicode-bidi: isolate;">WRSI</span> أعلى عبر موسم النمو.</p>
</div>

In [ ]:
tiaret_coords = {'x': 141143, 'y': 4244081}

tunis_wrsi = ds_wrsi.WRSI.sel(**tunis_coords, method = 'nearest').values
tiaret_wrsi = ds_wrsi.WRSI.sel(**tiaret_coords, method = 'nearest').values

pyplot.plot(date_labels[t:], tunis_wrsi, label = 'Tunis')
pyplot.plot(date_labels[t:], tiaret_wrsi, label = 'Tiaret')
pyplot.legend()
pyplot.show()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><hr></p><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مراجع</h2><ul><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">Fan, J., McConkey, B., Wang, H., &amp; Janzen, H. (2016). Root distribution by depth for temperate agricultural crops. Field Crops Research, 189, 68–74. <span dir="ltr" style="unicode-bidi: isolate;">https://doi.org/10.1016/j.fcr.2016.02.013</span></li><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">FAO 56, Table 19, &quot;Typical soil water characteristics for different soil types,&quot; <span dir="ltr" style="unicode-bidi: isolate;">https://www.fao.org/4/X0490E/x0490e0c.htm#soil%20evaporation%20reduction%20coefficient%20(kr)</span></li><li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">FAO 56, Table 22, &quot;Ranges of maximum effective rooting depth...for common crops,&quot; <span dir="ltr" style="unicode-bidi: isolate;">https://www.fao.org/4/X0490E/x0490e0e.htm#readily%20available%20water%20(raw)</span></li></ul>
</div>